In [75]:
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
import re
import pytz
!pip install python-docx
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import re 
from dateutil.relativedelta import relativedelta

In [76]:
import os
import pandas as pd

directory_path = '/Users/xiechen/Desktop/Affiliate'
files = os.listdir(directory_path)
print(files)

['.DS_Store', 'Agenda_Template.docx', '0601_0612.xlsx', '0520_0531.xlsx', '0510-0708.xlsx', 'Auto_Agenda.ipynb', '~$enda_Template.docx', 'Agenda_Template_Updated.docx', '~$enda 623-630.docx', '~$enda_Template_Updated.docx', 'milk_order.csv', 'Agenda', 'Yuanpeng_agenda.ipynb', '0710-0807.xlsx', 'order.csv']


In [77]:
'''
# Load the Excel files, skipping the first two rows
file1 = '/Users/xiechen/Desktop/Affiliate/0510-0708.xlsx'
file2 = '/Users/xiechen/Desktop/Affiliate/0710-0807.xlsx'

# Read the data from the Excel files into DataFrames, skipping the first two rows
df1 = pd.read_excel(file1, skiprows=2)
df2 = pd.read_excel(file2, skiprows=2)

# Display the column names to confirm they are correctly read
print("Columns in 0510-0708.xlsx:", df1.columns)
print("Columns in 0710-0807.xlsx:", df2.columns)

# Inspect the first few rows of each DataFrame
print("First few rows of df1:\n", df1.head())
print("First few rows of df2:\n", df2.head())

# Remove duplicates from df1 based on the 'Video ID' column if they exist in df2
df1_cleaned = df1[~df1['Video ID'].isin(df2['Video ID'])]

# Concatenate the cleaned df1 with df2
df_combined = pd.concat([df1_cleaned, df2])

# Save the merged DataFrame to a new Excel file
df_combined.to_excel('/Users/xiechen/Desktop/Affiliate/merged_video_performance.xlsx', index=False)
print("The data has been successfully merged and saved to 'merged_data.xlsx'.")
'''

'\n# Load the Excel files, skipping the first two rows\nfile1 = \'/Users/xiechen/Desktop/Affiliate/0510-0708.xlsx\'\nfile2 = \'/Users/xiechen/Desktop/Affiliate/0710-0807.xlsx\'\n\n# Read the data from the Excel files into DataFrames, skipping the first two rows\ndf1 = pd.read_excel(file1, skiprows=2)\ndf2 = pd.read_excel(file2, skiprows=2)\n\n# Display the column names to confirm they are correctly read\nprint("Columns in 0510-0708.xlsx:", df1.columns)\nprint("Columns in 0710-0807.xlsx:", df2.columns)\n\n# Inspect the first few rows of each DataFrame\nprint("First few rows of df1:\n", df1.head())\nprint("First few rows of df2:\n", df2.head())\n\n# Remove duplicates from df1 based on the \'Video ID\' column if they exist in df2\ndf1_cleaned = df1[~df1[\'Video ID\'].isin(df2[\'Video ID\'])]\n\n# Concatenate the cleaned df1 with df2\ndf_combined = pd.concat([df1_cleaned, df2])\n\n# Save the merged DataFrame to a new Excel file\ndf_combined.to_excel(\'/Users/xiechen/Desktop/Affiliate/merge

In [79]:
import pandas as pd
from datetime import datetime, timedelta
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
import numpy as np
import re
from dateutil.relativedelta import relativedelta

# Function to calculate previous date range 
def calculate_previous_date_range(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
    end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
    
    date_diff = end_date - start_date
    
    if date_diff.days == 6:  # Weekly
        previous_start_date = start_date - timedelta(weeks=1)
        previous_end_date = end_date - timedelta(weeks=1)
    elif date_diff.days == 14:  # (15 days)
        previous_start_date = start_date - timedelta(days=15)
        previous_end_date = end_date - timedelta(days=15)
    elif date_diff.days >= 28 and date_diff.days <= 31:  # Monthly 
        previous_start_date = start_date - relativedelta(months=1)
        previous_end_date = end_date - relativedelta(months=1)
    else:  
        previous_start_date = start_date - date_diff
        previous_end_date = end_date - date_diff
    
    return previous_start_date.strftime('%Y-%m-%d %H:%M:%S'), previous_end_date.strftime('%Y-%m-%d %H:%M:%S')

# Function to filter date range
def filter_date_range(df, date_column, start_date, end_date):
    df[date_column] = pd.to_datetime(df[date_column])
    mask = (df[date_column] >= start_date) & (df[date_column] <= end_date)
    return df.loc[mask]

# Clean and convert GMV column to numeric
def clean_gmv_column(df, column_name):
    df[column_name] = pd.to_numeric(df[column_name].replace('[\$,]', '', regex=True), errors='coerce')
    return df

# Format values with thousands separator
def format_value(value):
    if isinstance(value, (int, float, np.integer, np.floating)):
        return f"{value:,.0f}"
    return value

def format_percentage(value):
    if value >= 0:
        return f"increase by {value:.2f}%"
    else:
        return f"decrease by {abs(value):.2f}%"


def bold_percentage(paragraph):
    for run in paragraph.runs:
        text = run.text
        new_text = []
        start = 0
        for match in re.finditer(r'\d+(\.\d+)?%', text):
            end = match.start()
            if start < end:
                new_text.append((text[start:end], False))
            new_text.append((match.group(), True))
            start = match.end()
        if start < len(text):
            new_text.append((text[start:], False))

        # Clear the original text in the run
        run.clear()

        # Add the parts with or without bold formatting
        for part, bold in new_text:
            new_run = paragraph.add_run(part)
            new_run.bold = bold
            new_run.font.name = run.font.name
            new_run.font.size = run.font.size



# Function to calculate GMV for a DataFrame
def calculate_gmv(df, column_name='Video Revenue ($)'):
    return df[column_name].sum()

# Function to get user time period input 
def get_date_input(prompt):
    while True:
        date_str = input(prompt)
        try:
            datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
            return date_str
        except ValueError:
            print("Incorrect format, please enter the date in the format 'YYYY-MM-DD HH:MM:SS'.")

start_date = get_date_input("Enter the start date (YYYY-MM-DD HH:MM:SS): ")
end_date = get_date_input("Enter the end date (YYYY-MM-DD HH:MM:SS): ")

# Load data
previous_data = '/Users/xiechen/Desktop/Affiliate/0520_0531.xlsx'
now_data = '/Users/xiechen/Desktop/Affiliate/0601_0612.xlsx'
order_data = '/Users/xiechen/Desktop/Affiliate/milk_order.csv'
df_previous = pd.read_excel(previous_data, header=2)
df_now = pd.read_excel(now_data, header=2)
df_order = pd.read_csv(order_data)

# Clean and convert GMV column to numeric
df_previous = clean_gmv_column(df_previous, 'Video Revenue ($)')
df_now = clean_gmv_column(df_now, 'Video Revenue ($)')

# Convert the 'Time' column to datetime
df_previous['Time'] = pd.to_datetime(df_previous['Time'], errors='coerce')
df_now['Time'] = pd.to_datetime(df_now['Time'], errors='coerce')

# Filter the data for the specified date ranges using the 'Time' column
filtered_df_previous = filter_date_range(df_previous, 'Time', previous_start_date, previous_end_date)
filtered_df_now = filter_date_range(df_now, 'Time', start_date, end_date)

# Calculate GMV 
gmv_now = calculate_gmv(df_now)
gmv_previous = calculate_gmv(df_previous)

# Calculate VV
now_vv = df_now['VV'].sum()
previous_vv = df_previous['VV'].sum()
percentage_change_vv = ((now_vv - previous_vv) / previous_vv * 100) if previous_vv != 0 else 'N/A'
formatted_now_vv = format_value(now_vv)
formatted_previous_vv = format_value(previous_vv)
formatted_percentage_change_vv = format_percentage(percentage_change_vv) if percentage_change_vv != 'N/A' else 'N/A'

# Calculate product impression
now_product_impression = df_now['Product Impressions'].sum()
previous_product_impression = df_previous['Product Impressions'].sum()
percentage_change_product_impression = (
    (now_product_impression - previous_product_impression) / previous_product_impression * 100
) if previous_product_impression != 0 else 'N/A'
formatted_now_product_impression = format_value(now_product_impression)
formatted_previous_product_impression = format_value(previous_product_impression)
formatted_percentage_change_product_impression = format_percentage(percentage_change_product_impression) if percentage_change_product_impression != 'N/A' else 'N/A'

# Calculate CTR
df_now['Calculated CTR'] = df_now['Product Clicks'] / df_now['Product Impressions']
average_ctr_now = df_now['Calculated CTR'].mean()
formatted_average_ctr_now = f"{average_ctr_now * 100:.2f}%" if not np.isnan(average_ctr_now) else 'N/A'
df_previous['Calculated CTR'] = df_previous['Product Clicks'] / df_previous['Product Impressions']
average_ctr_previous = df_previous['Calculated CTR'].mean()
formatted_average_ctr_previous = f"{average_ctr_previous * 100:.2f}%" if not np.isnan(average_ctr_previous) else 'N/A'
percentage_change_ctr = (
    (average_ctr_now - average_ctr_previous) / average_ctr_previous * 100
) if average_ctr_previous != 0 else 'N/A'
formatted_percentage_change_ctr = format_percentage(percentage_change_ctr) if percentage_change_ctr != 'N/A' else 'N/A'

# Get unique products
unique_product = df_now['Products'].nunique()
new_product = filtered_df_now['Products'].nunique()

filtered_df_now = filtered_df_now.copy()

# Create the 'Engagement' column in the now dataset if it doesn't already exist
if 'Engagement' not in filtered_df_now.columns:
    filtered_df_now['Engagement'] = (
        filtered_df_now['Likes'] + 
        filtered_df_now['Shares'] + 
        filtered_df_now['Comments']
    )

# Create the 'Link' column in the now dataset
filtered_df_now['Link'] = (
    'www.tiktok.com/' + '@' + filtered_df_now['Creator name'] +
    '/video/' + filtered_df_now['Video ID'].astype(str)
)

# Sort the filtered DataFrame by 'Video Revenue ($)' in descending order and select the top 3 videos
filtered_top_videos = filtered_df_now.sort_values(by='Video Revenue ($)', ascending=False).head(3)

# Ensure that the 'Engagement' column exists in df_now
if 'Engagement' not in df_now.columns:
    df_now['Engagement'] = (
        df_now['Likes'] + 
        df_now['Shares'] + 
        df_now['Comments']
    )

if 'Engagement' not in df_previous.columns:
    df_previous['Engagement'] = (
        df_previous['Likes'] + 
        df_previous['Shares'] + 
        df_previous['Comments']
    )

# Sort by 'Video Revenue ($)'
historical_top_videos = df_now.sort_values(by='Video Revenue ($)', ascending=False).head(3)

# Create the 'Engagement' column in the historical dataset
if 'Engagement' not in historical_top_videos.columns:
    historical_top_videos['Engagement'] = (
        historical_top_videos['Likes'] + 
        historical_top_videos['Shares'] + 
        historical_top_videos['Comments']
    )

# Calculate the sums of 'Engagement' for now and previous datasets
now_engagement = df_now['Engagement'].sum()
previous_engagement = df_previous['Engagement'].sum()
percentage_change_engagement = (
    (now_engagement - previous_engagement) / previous_engagement * 100
) if previous_engagement != 0 else 'N/A'
formatted_now_engagement = format_value(now_engagement)
formatted_previous_engagement = format_value(previous_engagement)
formatted_percentage_change_engagement = format_percentage(percentage_change_engagement) if percentage_change_engagement != 'N/A' else 'N/A'

# Calculate the engagement rate
total_engagement = df_now['Likes'].sum() + df_now['Shares'].sum() + df_now['Comments'].sum()
total_views = df_now['VV'].sum()
engagement_rate = (total_engagement / total_views * 100) if total_views != 0 else 'N/A'
formatted_engagement_rate = f"{engagement_rate:.2f}%" if engagement_rate != 'N/A' else 'N/A'


# Create the 'Link' column in the historical dataset
historical_top_videos['Link'] = (
    'www.tiktok.com/' + '@' + historical_top_videos['Creator name'] +
    '/video/' + historical_top_videos['Video ID'].astype(str)
)

# Order metrics
df_order['Paid Time'] = df_order['Paid Time'].str.strip()
df_order['Paid Time'] = pd.to_datetime(df_order['Paid Time'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

# Calculate unique products name
filtered_df_order = filter_date_range(df_order, 'Paid Time', start_date, end_date)
unique_product_count = df_order['Product Name'].nunique()

#Calculate number products
total_product = len(df_order)
filtered_product = len(filtered_df_order)

# New product in the period
new_product_count = filtered_df_order['Product Name'].nunique()

# Products list
product_counts = filtered_df_order['Product Name'].value_counts()
product_list = "\n".join([f"• {product}: {count}" for product, count in product_counts.items()])


# Add GMV comparison result to the placeholders
placeholders = {
    '{start_date}': pd.to_datetime(start_date).strftime('%Y-%m-%d'),
    '{end_date}': pd.to_datetime(end_date).strftime('%Y-%m-%d'),
    '{total_gmv}': f"${format_value(gmv_now)}",
    '{gmv_comparison}': format_percentage((gmv_now - gmv_previous) / gmv_previous * 100) if gmv_previous != 0 else 'N/A',  
    '{unique_product}': format_value(total_product),
    '{new_product}': format_value(filtered_product),
    '{total_posts}': format_value(len(filtered_df_now)),
    '{percentage_change}': format_percentage((len(filtered_df_now) - len(filtered_df_previous)) / len(filtered_df_previous) * 100 if len(filtered_df_previous) > 0 else 0),
    '{total_vv}': format_value(filtered_df_now['VV'].sum()),
    '{total_engagement}': format_value(filtered_df_now['Engagement'].sum()),
    '{total_posts_with_vv}': format_value(len(df_now)),
    '{total_all_vv}': format_value(df_now['VV'].sum()),
    '{total_all_engagement}': format_value(df_now['Engagement'].sum()),
    '{vv_percentage_change}': formatted_percentage_change_vv,
    '{now_product_impression}': formatted_now_product_impression,
    '{product_impression_percentage_change}': formatted_percentage_change_product_impression,
    '{engagement_percentage_change}': formatted_percentage_change_engagement,
    '{engagement_rate}': formatted_engagement_rate,
    '{average_ctr_now}': formatted_average_ctr_now,
    '{ctr_percentage_change}': formatted_percentage_change_ctr,
    '{Product List}': product_list
}


# # Top 3 based on 'Video Revenue ($)'
placeholders.update({
    '{video_1_creator}': filtered_top_videos.iloc[0]['Creator name'] if len(filtered_top_videos) > 0 else '',
    '{video_1_products}': filtered_top_videos.iloc[0]['Products'] if len(filtered_top_videos) > 0 else '',
    '{video_1_vv}': format_value(filtered_top_videos.iloc[0]['VV']) if len(filtered_top_videos) > 0 else '',
    '{video_1_revenue}': f"${format_value(filtered_top_videos.iloc[0]['Video Revenue ($)'])}" if len(filtered_top_videos) > 0 else '',
    '{video_1_engagement}': format_value(filtered_top_videos.iloc[0]['Engagement']) if len(filtered_top_videos) > 0 else '',
    '{video_1_CTR}': format_value(filtered_top_videos.iloc[0]['CTR']) if len(filtered_top_videos) > 0 else '',
    '{video_1_link}': filtered_top_videos.iloc[0]['Link'] if len(filtered_top_videos) > 0 else '',
    
    '{video_2_creator}': filtered_top_videos.iloc[1]['Creator name'] if len(filtered_top_videos) > 1 else '',
    '{video_2_products}': filtered_top_videos.iloc[1]['Products'] if len(filtered_top_videos) > 1 else '',
    '{video_2_vv}': format_value(filtered_top_videos.iloc[1]['VV']) if len(filtered_top_videos) > 1 else '',
    '{video_2_revenue}': f"${format_value(filtered_top_videos.iloc[1]['Video Revenue ($)'])}" if len(filtered_top_videos) > 1 else '',
    '{video_2_engagement}': format_value(filtered_top_videos.iloc[1]['Engagement']) if len(filtered_top_videos) > 1 else '',
    '{video_2_CTR}': format_value(filtered_top_videos.iloc[1]['CTR']) if len(filtered_top_videos) > 0 else '',
    '{video_2_link}': filtered_top_videos.iloc[1]['Link'] if len(filtered_top_videos) > 1 else '',
    
    '{video_3_creator}': filtered_top_videos.iloc[2]['Creator name'] if len(filtered_top_videos) > 2 else '',
    '{video_3_products}': filtered_top_videos.iloc[2]['Products'] if len(filtered_top_videos) > 2 else '',
    '{video_3_vv}': format_value(filtered_top_videos.iloc[2]['VV']) if len(filtered_top_videos) > 2 else '',
    '{video_3_revenue}': f"${format_value(filtered_top_videos.iloc[2]['Video Revenue ($)'])}" if len(filtered_top_videos) > 2 else '',
    '{video_3_engagement}': format_value(filtered_top_videos.iloc[2]['Engagement']) if len(filtered_top_videos) > 2 else '',
    '{video_3_CTR}': format_value(filtered_top_videos.iloc[2]['CTR']) if len(filtered_top_videos) > 0 else '',
    '{video_3_link}': filtered_top_videos.iloc[2]['Link'] if len(filtered_top_videos) > 2 else '', 
    
    '{historical_video_1_creator}': historical_top_videos.iloc[0]['Creator name'] if len(historical_top_videos) > 0 else '',
    '{historical_video_1_products}': historical_top_videos.iloc[0]['Products'] if len(historical_top_videos) > 0 else '',
    '{historical_video_1_vv}': format_value(historical_top_videos.iloc[0]['VV']) if len(historical_top_videos) > 0 else '',
    '{historical_video_1_revenue}': f"${format_value(historical_top_videos.iloc[0]['Video Revenue ($)'])}" if len(historical_top_videos) > 0 else '',
    '{historical_video_1_engagement}': format_value(historical_top_videos.iloc[0]['Engagement']) if len(historical_top_videos) > 0 else '',
    '{historical_video_1_CTR}': historical_top_videos.iloc[0]['CTR'] if len(historical_top_videos) > 0 else '',
    '{historical_video_1_link}': historical_top_videos.iloc[0]['Link'] if len(historical_top_videos) > 0 else '',
    
    '{historical_video_2_creator}': historical_top_videos.iloc[1]['Creator name'] if len(historical_top_videos) > 1 else '',
    '{historical_video_2_products}': historical_top_videos.iloc[1]['Products'] if len(historical_top_videos) > 1 else '',
    '{historical_video_2_vv}': format_value(historical_top_videos.iloc[1]['VV']) if len(historical_top_videos) > 1 else '',
    '{historical_video_2_revenue}': f"${format_value(historical_top_videos.iloc[1]['Video Revenue ($)'])}" if len(historical_top_videos) > 1 else '',
    '{historical_video_2_engagement}': format_value(historical_top_videos.iloc[1]['Engagement']) if len(historical_top_videos) > 1 else '',
    '{historical_video_2_CTR}': historical_top_videos.iloc[1]['CTR'] if len(historical_top_videos) > 0 else '',
    '{historical_video_2_link}': historical_top_videos.iloc[1]['Link'] if len(historical_top_videos) > 1 else '',
    
    '{historical_video_3_creator}': historical_top_videos.iloc[2]['Creator name'] if len(historical_top_videos) > 2 else '',
    '{historical_video_3_products}': historical_top_videos.iloc[2]['Products'] if len(historical_top_videos) > 2 else '',
    '{historical_video_3_vv}': format_value(historical_top_videos.iloc[2]['VV']) if len(historical_top_videos) > 2 else '',
    '{historical_video_3_revenue}': f"${format_value(historical_top_videos.iloc[2]['Video Revenue ($)'])}" if len(historical_top_videos) > 2 else '',
    '{historical_video_3_engagement}': format_value(historical_top_videos.iloc[2]['Engagement']) if len(historical_top_videos) > 2 else '',
    '{historical_video_3_CTR}': historical_top_videos.iloc[2]['CTR'] if len(historical_top_videos) > 0 else '',
    '{historical_video_3_link}': historical_top_videos.iloc[2]['Link'] if len(historical_top_videos) > 2 else '',
    
})

# Sort by 'VV'
vv_filtered_top_videos = filtered_top_videos.sort_values(by='VV', ascending=False).head(3)
vv_historical_top_videos = historical_top_videos.sort_values(by='VV', ascending=False).head(3)

# Top 3 based on 'VV'
placeholders.update({
    '{vv_video_1_creator}': vv_filtered_top_videos.iloc[0]['Creator name'] if len(vv_filtered_top_videos) > 0 else '',
    '{vv_video_1_products}': vv_filtered_top_videos.iloc[0]['Products'] if len(vv_filtered_top_videos) > 0 else '',
    '{vv_video_1_vv}': format_value(vv_filtered_top_videos.iloc[0]['VV']) if len(vv_filtered_top_videos) > 0 else '',
    '{vv_video_1_revenue}': f"${format_value(vv_filtered_top_videos.iloc[0]['Video Revenue ($)'])}" if len(vv_filtered_top_videos) > 0 else '',
    '{vv_video_1_engagement}': format_value(vv_filtered_top_videos.iloc[0]['Engagement']) if len(vv_filtered_top_videos) > 0 else '',
    '{vv_video_1_CTR}': format_value(vv_filtered_top_videos.iloc[0]['CTR']) if len(vv_filtered_top_videos) > 0 else '',
    '{vv_video_1_link}': vv_filtered_top_videos.iloc[0]['Link'] if len(vv_filtered_top_videos) > 0 else '',
    
    '{vv_video_2_creator}': vv_filtered_top_videos.iloc[1]['Creator name'] if len(vv_filtered_top_videos) > 1 else '',
    '{vv_video_2_products}': vv_filtered_top_videos.iloc[1]['Products'] if len(vv_filtered_top_videos) > 1 else '',
    '{vv_video_2_vv}': format_value(vv_filtered_top_videos.iloc[1]['VV']) if len(vv_filtered_top_videos) > 1 else '',
    '{vv_video_2_revenue}': f"${format_value(vv_filtered_top_videos.iloc[1]['Video Revenue ($)'])}" if len(vv_filtered_top_videos) > 1 else '',
    '{vv_video_2_engagement}': format_value(vv_filtered_top_videos.iloc[1]['Engagement']) if len(vv_filtered_top_videos) > 1 else '',
    '{vv_video_2_CTR}': format_value(vv_filtered_top_videos.iloc[1]['CTR']) if len(vv_filtered_top_videos) > 0 else '',
    '{vv_video_2_link}': vv_filtered_top_videos.iloc[1]['Link'] if len(vv_filtered_top_videos) > 1 else '',
    
    '{vv_video_3_creator}': vv_filtered_top_videos.iloc[2]['Creator name'] if len(vv_filtered_top_videos) > 2 else '',
    '{vv_video_3_products}': vv_filtered_top_videos.iloc[2]['Products'] if len(vv_filtered_top_videos) > 2 else '',
    '{vv_video_3_vv}': format_value(vv_filtered_top_videos.iloc[2]['VV']) if len(vv_filtered_top_videos) > 2 else '',
    '{vv_video_3_revenue}': f"${format_value(vv_filtered_top_videos.iloc[2]['Video Revenue ($)'])}" if len(vv_filtered_top_videos) > 2 else '',
    '{vv_video_3_engagement}': format_value(vv_filtered_top_videos.iloc[2]['Engagement']) if len(vv_filtered_top_videos) > 2 else '',
    '{vv_video_3_CTR}': format_value(vv_filtered_top_videos.iloc[2]['CTR']) if len(vv_filtered_top_videos) > 0 else '',
    '{vv_video_3_link}': vv_filtered_top_videos.iloc[2]['Link'] if len(vv_filtered_top_videos) > 2 else '',

    '{vv_historical_video_1_creator}': vv_historical_top_videos.iloc[0]['Creator name'] if len(vv_historical_top_videos) > 0 else '',
    '{vv_historical_video_1_products}': vv_historical_top_videos.iloc[0]['Products'] if len(vv_historical_top_videos) > 0 else '',
    '{vv_historical_video_1_vv}': format_value(vv_historical_top_videos.iloc[0]['VV']) if len(vv_historical_top_videos) > 0 else '',
    '{vv_historical_video_1_revenue}': f"${format_value(vv_historical_top_videos.iloc[0]['Video Revenue ($)'])}" if len(vv_historical_top_videos) > 0 else '',
    '{vv_historical_video_1_engagement}': format_value(vv_historical_top_videos.iloc[0]['Engagement']) if len(vv_historical_top_videos) > 0 else '',
    '{vv_historical_video_1_CTR}': format_value(vv_historical_top_videos.iloc[0]['CTR']) if len(vv_historical_top_videos) > 0 else '',
    '{vv_historical_video_1_link}': vv_historical_top_videos.iloc[0]['Link'] if len(vv_historical_top_videos) > 0 else '',

    '{vv_historical_video_2_creator}': vv_historical_top_videos.iloc[1]['Creator name'] if len(vv_historical_top_videos) > 1 else '',
    '{vv_historical_video_2_products}': vv_historical_top_videos.iloc[1]['Products'] if len(vv_historical_top_videos) > 1 else '',
    '{vv_historical_video_2_vv}': format_value(vv_historical_top_videos.iloc[1]['VV']) if len(vv_historical_top_videos) > 1 else '',
    '{vv_historical_video_2_revenue}': f"${format_value(vv_historical_top_videos.iloc[1]['Video Revenue ($)'])}" if len(vv_historical_top_videos) > 1 else '',
    '{vv_historical_video_2_engagement}': format_value(vv_historical_top_videos.iloc[1]['Engagement']) if len(vv_historical_top_videos) > 1 else '',
    '{vv_historical_video_2_CTR}': format_value(vv_historical_top_videos.iloc[1]['CTR']) if len(vv_historical_top_videos) > 0 else '',
    '{vv_historical_video_2_link}': vv_historical_top_videos.iloc[1]['Link'] if len(vv_historical_top_videos) > 1 else '',

    '{vv_historical_video_3_creator}': vv_historical_top_videos.iloc[2]['Creator name'] if len(vv_historical_top_videos) > 2 else '',
    '{vv_historical_video_3_products}': vv_historical_top_videos.iloc[2]['Products'] if len(vv_historical_top_videos) > 2 else '',
    '{vv_historical_video_3_vv}': format_value(vv_historical_top_videos.iloc[2]['VV']) if len(vv_historical_top_videos) > 2 else '',
    '{vv_historical_video_3_revenue}': f"${format_value(vv_historical_top_videos.iloc[2]['Video Revenue ($)'])}" if len(vv_historical_top_videos) > 2 else '',
    '{vv_historical_video_3_engagement}': format_value(vv_historical_top_videos.iloc[2]['Engagement']) if len(vv_historical_top_videos) > 2 else '',
    '{vv_historical_video_3_CTR}': format_value(vv_historical_top_videos.iloc[2]['CTR']) if len(vv_historical_top_videos) > 0 else '',
    '{vv_historical_video_3_link}': vv_historical_top_videos.iloc[2]['Link'] if len(vv_historical_top_videos) > 2 else ''
})


doc = Document('/Users/xiechen/Desktop/Affiliate/Agenda_Template.docx')
for paragraph in doc.paragraphs:
    replace_placeholder(paragraph, placeholders)
    set_font(paragraph)

# Replace placeholders in table cells and set the designated font
for table in doc.tables:
    for row in table.rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                replace_placeholder(paragraph, placeholders)
                set_font(paragraph)

# Save the modified document
output_path = '/Users/xiechen/Desktop/Affiliate/Agenda_Template_Updated.docx'
doc.save(output_path)
print(f"Document saved to {output_path}")

Enter the start date (YYYY-MM-DD HH:MM:SS):  2024-06-01 00:00:00
Enter the end date (YYYY-MM-DD HH:MM:SS):  2024-06-12 23:59:59


Document saved to /Users/xiechen/Desktop/Affiliate/Agenda_Template_Updated.docx
